In [85]:
import tensorflow as tf
import re
from nltk.corpus import stopwords
import nltk
stop_words = stopwords.words('english')
from collections import defaultdict
import time
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

In [41]:
def text_processing(line):
    s=re.sub(r'<.+?>', '', line)
    s =re.sub("\d+", "", s)
    s=s.replace('@en .', '')
    s= re.sub(r'[\(\)\"\:\.\$\&\'\#\%\[\]\+\!\?\-\\]','',s)
    s=s.replace('\n','')
    s=s.replace(';','')
    s=re.sub(r"\b[a-zA-Z]\b", "", s)
    s=s.replace('u\2014',' ')
    s=re.sub(r'\([^)]*\)', '', s)
    return s

In [4]:
from gensim.models import Word2Vec
f=open('/home/kapil/Documents/IISc_sem/Machine_Learning_Large_Datasets/Assignment2/DBPedia.full/full_train.txt','r')
lines=f.readlines()
Words=[]
for line in lines:
    List1=[]
    line=text_processing(line)
    line=line.lower()
    for word in line.split('\t')[1].split():  
        word=re.sub(r"[^a-zA-Z]","",word)
        if word not in stop_words:
            List1.append(word.rstrip())
    Words.append(List1)


In [5]:
model = Word2Vec(Words)

In [6]:
Titles=set()
total_ex=0
for line in lines:
    line=line.lower()
    line=text_processing(line)
    for title in line.split('\t')[0].split(','):
        Titles.add(title.rstrip())
        total_ex+=1

In [7]:
import numpy as np
print(total_ex)

298176


In [8]:
b = dict(enumerate(Titles))
inv_map = {v: k for k, v in b.items()}
target=np.zeros((total_ex, 50))
counter=0
for line in lines:
    line=text_processing(line).lower()
    for title in line.split('\t')[0].split(','):
        target[counter,inv_map[title.rstrip()]]=1
        counter+=1

In [9]:
def Softmax(W,emb,n,k,Y):
    sum1=0
    for k1 in range(50):
        sum1+=np.exp(np.dot(W[:,k1],emb))
    Y[n,k]=np.exp(np.dot(W[:,k],emb))/sum1
    return Y

In [11]:
W=np.random.randn(100, 50)
print(W[:,1].shape)
print(total_ex)

(100,)
298176


In [16]:
W=np.random.randn(100, 50)
Y=np.zeros((total_ex,50))
f=open('/home/kapil/Documents/IISc_sem/Machine_Learning_Large_Datasets/Assignment2/DBPedia.full/full_train.txt','r')
lines=f.readlines()
mue=0.0001
lmda=0.1

num_epoch=10
from progressbar import ProgressBar
pbar = ProgressBar()
start_time = time.time()
for epoch in pbar(range(num_epoch)):
    n=0
    for line in lines:
        Doc_rep=0
        count=0
        try:  
            line=text_processing(line).lower()
            for word in line.split('\t')[1].split():
                word=re.sub(r"[^a-zA-Z]","",word)
                if word not in stop_words:
                    try:
                        Doc_rep+=model[word.rstrip()]
                    except KeyError:
                        continue
                count+=1
            emb=Doc_rep/count
            for title in line.split('\t')[0].split(','):
                for i in range(50):
                    Y=Softmax(W,emb,n,i,Y)
                    W[:,i]=W[:,i]*(1-mue*lmda)-mue*(Y[n,i]-target[n,i])*emb 
                n+=1
        
        except ZeroDivisionError:
            continue
    print("--- %s seconds ---" % (time.time() - start_time))

/home/kapil/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


--- 2466.254831790924 seconds ---


--- 4865.425188779831 seconds ---


--- 7265.362658023834 seconds ---


--- 9695.706987380981 seconds ---


...

--- 14598.24015045166 seconds ---


--- 17335.245698451996 seconds ---


--- 19735.880929231644 seconds ---


--- 22177.845766067505 seconds ---


--- 24829.04036974907 seconds ---


In [64]:
Y1=np.zeros((total_ex,50))
def Softmax1(W,emb,n,k,Y1):
    sum1=0
    for k1 in range(50):
        sum1+=np.exp(np.dot(W[:,k1],emb))
    Y1[n,k]=np.exp(np.dot(W[:,k],emb))/sum1
    return Y1

In [82]:
f=open('/home/kapil/Documents/IISc_sem/Machine_Learning_Large_Datasets/Assignment2/DBPedia.full/full_test.txt','r')
lines=f.readlines()
n1=0
num_ex=0
correct=0
from progressbar import ProgressBar
pbar = ProgressBar()
for line in pbar(lines):
    num_ex+=1
    T1=[]
    line=text_processing(line).lower()
    Doc_rep1=np.zeros((100,))
    emb1=0
    count1=0
    try:
        for word in line.split('\t')[1].split():
            word=re.sub(r"[^a-zA-Z]","",word)
            if word not in stop_words:
                try:
                    Doc_rep1+=model[word.rstrip()]
                except KeyError:
                    continue
            count1+=1
        if Doc_rep1.all()==0:
            continue
        emb1=Doc_rep1/count1
        for t in line.split('\t')[0].split(','):
            T1.append(t.rstrip())
        for i in range(50):
            Y1=Softmax1(W,emb1,n1,i,Y1)      
        if b[list(Y1[n1,:]).index(max(list(Y1[n1,:])))] in T1:
            correct+=1
        n1+=1
    except ZeroDivisionError:
            continue

/home/kapil/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
100% |########################################################################|


In [83]:
print(correct/n1)

0.2143690615102517


In [84]:
#print(correct/n1)
#print(num_ex)
#print(lines[num_ex])
print(text_processing(lines[num_ex-1]).lower())
#Y1=Softmax1(W,emb1,n1,i,Y1)
emb2=0
count2=0
Doc_rep2=0
for word in text_processing(lines[num_ex-1]).lower().split('\t')[1].split():
    word=re.sub(r"[^a-zA-Z]","",word)
    if word not in stop_words:
        try:
            Doc_rep2+=model[word.rstrip()]
        except KeyError:
            continue
    count2+=1
emb2=Doc_rep2/count2
print(Doc_rep2.shape)
print(emb2)
print(model['directed'])
sum2=0
for k2 in range(50):
    sum2+=np.exp(np.dot(W[:,k2],emb2))
print(W[:,i])
print(np.exp(np.dot(W[:,i],emb2))/sum2)


indian_films 	  shanthi nivasa kannada ucbucbeucucaucbf ucaucbfucbucbeucb is   indian kannada film, directed by   bhargava and produced by   manik chand and   mohan the film stars anant nag, bharathi, vijayakashi and ramesh bhat in lead roles the film had musical score by  ranga rao
(100,)
[ 0.20119135 -0.23242852 -0.37620047  0.6432703   0.3805988  -0.14872755
 -0.02106177  0.220505    0.60288537 -0.8069491  -0.17061305 -0.51156706
  0.40369982 -0.6273393   0.79997253 -0.22948486 -0.5748347   0.88297635
  0.27585208  0.22398368  0.73144627  0.2317198  -0.23646776  0.33703753
  0.13311142  0.5658218   0.2748705  -0.3614643   0.42045656  0.5399151
  0.33698383 -0.06387739  0.45785668 -0.60800755 -0.31503665  0.50295
  0.360392   -0.13579127 -0.0717108  -0.3803081   0.9296617  -0.09707762
 -0.152438    0.4720538  -1.1060071   0.06770971  0.32260674  0.3079747
 -0.8669964  -0.5402027  -0.66552854  0.5199623   0.30389026  0.5060264
  0.13928896  0.29287788  0.58820206 -0.6667451  -0.946693

/home/kapil/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/home/kapil/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
